In [ ]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_contour,
    plot_slice,
    plot_edf,
    plot_rank,
    plot_pareto_front
)

#### 📦 1. Study 설정 및 로드

In [ ]:
# Study 정보 설정 (여기만 수정)
study_name = "vae_hparam_search"  # <- 스터디 이름
storage_path = "sqlite:///path/to/your/optuna.db"  # <- DB 경로

# Study 불러오기
study = optuna.load_study(study_name=study_name, storage=storage_path)

#### 🔍 2. Study 기본 요약

In [ ]:
# Study 요약
print(f"📊 Study name: {study.study_name}")
print(f"✅ Number of trials: {len(study.trials)}")
print(f"🎯 Optimization directions: {study.directions}")

is_multiobjective = len(study.directions) > 1
print(f"🎯 Multi-objective study: {is_multiobjective}")

if not is_multiobjective:
    best = study.best_trial
    print(f"🥇 Best trial (ID: {best.number}) - Objectives: {best.value}")
    for key, value in best.params.items():
        print(f"  - {key}: {value}")
else:
    print("🌟 Multi-objective study: showing Pareto-optimal trials")
    pareto_trials = study.best_trials
    for i, trial in enumerate(pareto_trials):
        print(f"Trial {trial.number} - Objectives: {trial.values}")
        for key, value in trial.params.items():
            print(f"  - {key}: {value}")
        print()

#### 📋 3. 간소화된 Trials 데이터프레임 보기

In [ ]:
# 원하는 필드만 표시
df_simple = study.trials_dataframe(attrs=("number", "value", "params", "state"))
df_simple.head()


#### 💾 4. Trial 데이터 CSV로 저장

In [ ]:
df_trials = study.trials_dataframe()
df_trials.to_csv("optuna_trials_export.csv", index=False)
print("📁 Trials exported to optuna_trials_export.csv")

#### ❗ 5. 실패한 Trial 로그

In [ ]:
failed_trials = [t for t in study.trials if t.state.name != "COMPLETE"]
print(f"❗ Failed trials: {len(failed_trials)}")

for trial in failed_trials:
    print(f"Trial {trial.number} failed with state: {trial.state.name}")

#### 📈 6. 단일 목적/다목적 공통 시각화

In [ ]:
# plot_optimization_history, plot_param_importances
if is_multiobjective:
    for i in range(len(study.directions)):
        print(f"\n🎯 Objective {i} Optimization History")
        try:
            plot_optimization_history(study, target=lambda t: t.values[i], target_name=f"Objective {i}").show()
        except Exception as e:
            print(f"⚠️ plot_optimization_history for Objective {i} failed:", e)

        print(f"\n🎯 Objective {i} Param Importances")
        try:
            plot_param_importances(study, target=lambda t: t.values[i], target_name=f"Objective {i}").show()
        except Exception as e:
            print(f"⚠️ plot_param_importances for Objective {i} failed:", e)
else:
    plot_optimization_history(study).show()
    plot_param_importances(study).show()

In [ ]:
# plot_parallel_coordinate, plot_contour, plot_slice
if is_multiobjective:
    for i in range(len(study.directions)):
        print(f"\n🎯 Parallel Coordinate Plot for Objective {i}")
        plot_parallel_coordinate(
            study, target=lambda t: t.values[i], target_name=f"Objective {i}"
        ).show()

        print(f"\n🎯 Contour Plot for Objective {i}")
        plot_contour(
            study, target=lambda t: t.values[i], target_name=f"Objective {i}"
        ).show()

        print(f"\n🎯 Slice Plot for Objective {i}")
        plot_slice(
            study, target=lambda t: t.values[i], target_name=f"Objective {i}"
        ).show()
else:
    plot_parallel_coordinate(study).show()
    plot_contour(study).show()
    plot_slice(study).show()

In [ ]:
# EDF: Empirical Distribution Function plot & Rank plot
if is_multiobjective:
    for i in range(len(study.directions)):
        print(f"\n📈 EDF for Objective {i}")
        plot_edf(
            study,
            target=lambda t: t.values[i],
            target_name=f"Objective {i}"
        ).show()

        print(f"📊 Rank plot for Objective {i}")
        plot_rank(
            study,
            target=lambda t: t.values[i],
            target_name=f"Objective {i}"
        ).show()
else:
    print("\n📈 EDF (Single Objective)")
    plot_edf(study).show()

    print("📊 Rank plot (Single Objective)")
    plot_rank(study).show()

#### 🌐 7. Pareto Front 시각화 (2D만 지원)

In [ ]:
if is_multiobjective and len(study.directions) == 2 and len(pareto_trials) > 0:
    pareto_df = pd.DataFrame([trial.values for trial in pareto_trials], columns=["Objective 1", "Objective 2"])
    pareto_df["Trial"] = [t.number for t in pareto_trials]

    fig = px.scatter(
        pareto_df,
        x="Objective 1",
        y="Objective 2",
        text="Trial",
        title="Pareto Front (2D)",
        labels={"Objective 1": "Objective 1", "Objective 2": "Objective 2"}
    )
    fig.update_traces(textposition='top center')
    fig.show()
else:
    print("⚠️ Pareto front plotting is currently supported only for 2 objectives.")

#### 📊 8. Pareto Front 간 상관관계 분석

In [ ]:
if is_multiobjective and len(study.directions) == 2 and len(pareto_trials) > 0:
    pareto_df_full = pd.DataFrame([trial.values for trial in pareto_trials])
    pareto_df_full.columns = [f"Objective {i+1}" for i in range(len(study.directions))]

    plt.figure(figsize=(6, 4))
    sns.heatmap(pareto_df_full.corr(), annot=True, cmap="coolwarm")
    plt.title("Correlation between Objectives (Pareto Trials)")
    plt.tight_layout()
    plt.show()

#### 📌 9. Pareto Front 시각화 (Optuna 기본 제공)

In [ ]:
if len(study.directions) in [2, 3]:
    try:
        plot_pareto_front(study).show()
    except Exception as e:
        print("Pareto front plot failed:", e)
else:
    print("⚠️ plot_pareto_front only supports 2 or 3 objectives.")